This tutorial describes one particular intermediate representation used by the Devito Compiler: the Iteration/Expression Tree (IET), a special type of Abstract Syntax Tree.

# Part I - Top Down

Here, we investigate the IET of a simple ``Operator``.

First, let's describe a $domain$, and a $Function$ that will allow us to specify how such a domain gets modified. In particular, we will look at functions that change through $time$. 

Thus we need a function object with which we can build a timestepping scheme. For this purpose Devito provides so-called TimeData objects that encapsulate functions that are differentiable in space and time, which are derived from basic $SymPy$ functions. 

With this we can derive symbolic expressions for the backward derivatives in space directly via the `u.dxl` and `u.dyl` shorthand expressions (the l indicates "left" or backward differences) and the shorthand notation `u.dt` provided by TimeData objects to derive the forward derivative in time.



In [1]:
from devito import Eq, Grid, TimeFunction, Operator

grid = Grid(shape=(3, 3))
u = TimeFunction(name='u', grid=grid)
u.data

Data([[[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]],

      [[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]], dtype=float32)

Here, we have just declared a two-dimensional Domain with three coordinates at each dimension ($x$ and $y$). Each coordinate of such a discrete space will be holding a $real~value$.

As we can see, we can always access the values at each coordinate of the Domain. At this point, no modifications have been done to it yet. `u.data` give as a quick access over the values holded by each cell of such a Domain. `u.data[0]` holds the values in the grid at the "current" iteration time, given a time-step to be considered, whereas `u.data[1]` holds the values of `u` for the "current+1" time-step.

We can now create an `Operator` that will perform modifications onto our Domain according to $differential~ equations$ through a computational stencil. 
It means that those differential equations will be translated into finite differences that will be used to update the values at each coordinate of the space.
Such finite differences, or `expressions`, will be applied for specific ranges of `iterations` over the Domain. 

In [2]:
eq = Eq(u.forward, u+1)
op = Operator(eq)
op.args['expressions']

Eq(u(t + dt, x, y), u(t, x, y) + 1)